# A simple waveform demo

Select an audio file to explore. Use the tools on the left to navigate the waveform and click a button to play a portion of the waveform in your browser.

If running in a BinderHub instance instead of in a local notebook, it might be necessary to change the `default_url` value below.

In [ ]:
import os
import urllib
from bokeh_phon.models.audio_plot import AudioPlot
from phonlab.utils import dir2df
from bokeh.models import Button, Select
from bokeh.io import show, output_notebook
from bokeh.layouts import column
import parselmouth
import numpy as np
output_notebook()

In [ ]:
# The remote_jupyter_proxy_url function is required when running on a BinderHub instance.
# Change the default_url value to match the hostname of your instance after it has
# started. The current value is the most frequent result when launching from mybinder.org.
default_url = 'https://hub.gke.mybinder.org/'

def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    try:
        base_url = os.environ['EXTERNAL_URL']
    except KeyError:
        base_url = default_url
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url

In [ ]:
def myapp(doc):
    def load_wav_cb(attr, old, new):
        snd = parselmouth.Sound(new)
        samples = np.squeeze(snd.values).astype(np.float32)
        ap.fs = np.float32(snd.sampling_frequency)
        ap.wav.data_source.data = {
            'times': np.arange(len(samples)) / ap.fs,
            'samples': samples
        }
        ap.selbox.left = 0.0
        ap.selbox.right = 0.0
        ap.selbox.visible = False

    fdf = dir2df('..', fnpat='.*\.wav$', dirpat='resource')
    fdf['fpath'] = '../' + fdf.relpath.str.cat(fdf.fname, sep='/')
    options = [('', 'Choose an audio file')] + list(fdf.loc[:,['fpath', 'fname']].itertuples(index=False, name=None))
    fselect = Select(options=options)
    fselect.on_change('value', load_wav_cb)
    ap = AudioPlot(
        samples=np.array([0, 0]),
        fs=44100,
        # Remaining arguments are passed to Figure().
        plot_height=200,
        toolbar_location='left'
    )
    playallbtn = Button(label='Play all')
    playallbtn.js_on_event('button_click', ap.js_playall_cb)
    playselbtn = Button(label='Play selection')
    playselbtn.js_on_event('button_click', ap.js_playsel_cb)
    playxrbtn = Button(label='Play x range')
    playxrbtn.js_on_event('button_click', ap.js_playxr_cb)
    col = column(fselect, playallbtn, playselbtn, playxrbtn, ap)
    doc.add_root(col)
    return doc


# The notebook_url parameter is required when running in a BinderHub instance.
# If running a local notebook, omit that parameter.
#show(myapp)    # For running a local notebook
show(myapp, notebook_url=remote_jupyter_proxy_url)